<a href="https://colab.research.google.com/github/anish-mulay07/Quantitative-Stock-Price-Analysis-GPT-Investor/blob/main/gpt_investor_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Chat with Multiple PDF's Llama 2 + Pinecone + LangChain**

#**Step: 01: Install All the Required Packages**

In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_c

#**Step 2: Import All the Required Libraries**

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys

In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

#**Step 3: Load the PDF Files**

In [ ]:
# !mkdir pdfs

In [ ]:
# !gdown 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf
# !gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/GeorgiaTech Fintech Project/full-submission.txt", 'r') as file:
    # Read the entire content of the file
    data = file.read()
print((data))

from bs4 import BeautifulSoup

# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Extract text from the parsed HTML
cleaned_data = soup.get_text(separator='\n', strip=True)

# Print the cleaned text
print(len(cleaned_data))

with open('cleaned_data.txt', 'w') as file:
    # Write the cleaned text to the file
    file.write(cleaned_data)

print("Cleaned text has been written to 'cleaned_text.txt' file.")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6737070
Cleaned text has been written to 'cleaned_text.txt' file.


In [ ]:
print(len(cleaned_data))

6737070


In [ ]:

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1024,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents([data])

remove_indices = []
print("before cleaning", len(texts))

for index, text in enumerate(texts):
  texts[index] = text.page_content

for iterations in range(10):
  # print("iteration", iterations)
  for index, text in enumerate(texts):
      # Parse the HTML content
      soup = BeautifulSoup(text, 'html.parser')

      # Extract text from the parsed HTML
      cleaned_text = soup.get_text(separator='\n', strip=True)
      if len(cleaned_text) < 50:
        remove_indices.append(index)

      texts[index] = cleaned_text


  iterations+=1

print(len(texts))
print(max(remove_indices))

for index in sorted(remove_indices, reverse=True):
      texts[index] = 0

for index, text in enumerate(texts):
  if text == 0:
    del texts[index]

print("after cleaning", len(texts))


print(texts[0])
print(texts[1])
print(texts[1])
print(texts[1000])

before cleaning 9644


<ipython-input-9-4e802609ceef>:21: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


9644
9643
after cleaning 7584
lign:left;padding-left:12px;text-indent:-12px;font-size:9pt;">
Foreign exchange contracts
$
56,156
<br
clear="none"/>
$
1,049
<td style="vertical-align:bottom;background-color:#efefef;" rowspan="1"
clear="none"/>
$
1,049
<td style="vertical-align:bottom;background-color:#efefef;" rowspan="1"
style="font-family:inherit;font-size:10pt;">
Second Quarter
<font


In [ ]:
joined_texts = " "
print(len(joined_texts))
print(type(joined_texts))
for index, text in enumerate(texts):
  # print(type(text))

  joined_texts += str(text)

print(len(joined_texts))

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1024,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents([joined_texts])

print("lenght of texts: ",len(texts))

# print(texts[1000])
print(texts[1001].page_content)

1
<class 'str'>
1615076
lenght of texts:  1739
-Section 50
-Paragraph 5
-Subparagraph (aa)
-URI http://asc.fasb.org/extlink&oid=82897076&loc=d3e27232-111563
Reference 3: http://www.xbrl.org/2003/role/presentationRef
-Publisher FASB
-Name Accounting Standards Codification
-Topic 320
-SubTopic 10
-Section 45
-Paragraph 1
-URI http://asc.fasb.org/extlink&oid=82896770&loc=d3e26610-111562+ Details
Name:
us-gaap_AvailableForSaleSecurities
Namespace Prefix:
us-gaap_
Data Type:
xbrli:monetaryItemType
Balance Type:
debit
Period Type:
instant
X- Definition
Amount before tax of unrealized gain in accumulated other comprehensive income (AOCI) on investments in debt and equity securities classified as available-for-sale.
+ References
Reference 1: http://www.xbrl.org/2003/role/presentationRef
-Publisher FASB
-Name Accounting Standards Codification
-Topic 320
-SubTopic 10
-Section 50
-Paragraph 2
-Subparagraph (b)
-URI http://asc.fasb.org/extlink&oid=82897076&loc=d3e27161-111563
+ Details
Name:Name:


#**Step 04: Extract the text from the PDF's**

In [ ]:
# loader = PyPDFDirectoryLoader("pdfs")
# data = loader.load()

In [ ]:
# print(data)

#**Step 05: Split the Extracted Data into Text Chunks**

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
# docs = text_splitter.split_documents(data)

In [ ]:
# print(len(docs))

In [ ]:
# docs[3]

In [ ]:
# docs[4]

#**Step 06: Downlaod the Embeddings From the Hugging Face**

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query_result = embeddings.embed_query("Hello World")

In [ ]:
print("Length", len(query_result))

Length 384


#**Step 07: Initializing the Pinecone**

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '0e002073-f5f9-4dfc-8683-d95748185b9f')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-east-1')

In [ ]:
# # initialize pinecone
# pinecone.init(
#     api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#     environment=PINECONE_API_ENV  # next to api key in console
# )
# index_name = "fintech" # put in the name of your pinecone index here


from pinecone import Pinecone as PC
from pinecone import ServerlessSpec
import time
pc = PC(api_key="0e002073-f5f9-4dfc-8683-d95748185b9f")
spec = ServerlessSpec(cloud="aws", region="us-east-1")
# index_name = pc.Index("fintech3")
index_name = "fintech4"
# print(f"Index dimensionality: {index.dimension}")
# embedding_dim = 768
# index = pc.create_index(index_name, dimension=embedding_dim, spec=spec)


In [ ]:


# check if index already exists (it shouldn't if this is your first run)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=768,  # dimensionality of text-embed-3-small
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1739}},
 'total_vector_count': 1739}

In [ ]:
print(index)

In [ ]:
index = pc.Index(index_name)
vectors = index.query(vector = [], top_k = 5)


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 10 May 2024 04:44:23 GMT', 'Content-Type': 'application/json', 'Content-Length': '90', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '39', 'x-pinecone-request-id': '534542357373076966', 'x-envoy-upstream-service-time': '40', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Cannot provide both 'ID' and 'vector' at the same time","details":[]}


#**Step 08: Create Embeddings for each of the Text Chunk**

In [ ]:
# Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
# docsearch=Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

# from tqdm.auto import tqdm

# count = 0  # we'll use the count to create unique IDs
# batch_size = 32  # process everything in batches of 32
# for i in tqdm(range(0, len(texts), batch_size)):
#     # set end position of batch
#     i_end = min(i+batch_size, len(texts))
#     # get batch of lines and IDs
#     lines_batch = texts[i: i+batch_size]
#     ids_batch = [str(n) for n in range(i, i_end)]
#     # create embeddings
#     res = client.embeddings.create(input=lines_batch, model=MODEL)
#     embeds = [record.embedding for record in res.data]
#     # prep metadata and upsert batch
#     meta = [{'text': line} for line in lines_batch]
#     to_upsert = zip(ids_batch, embeds, meta)
#     # upsert to Pinecone
#     index.upsert(vectors=list(to_upsert))

# Create embeddings and store them in the Pinecone index
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

# Verify the dimensionality of the generated embeddings
sample_embedding = embeddings.embed_query(texts[0].page_content)
print(f"Embedding dimensionality: {len(sample_embedding)}")
for i, text in enumerate(texts):
    embedding = embeddings.embed_query(str(text.page_content))
    index.upsert([{"id": str(i), "values": embedding, "metadata": {"text": text.page_content}}])


#If you already have an index, you can load it like this

In [ ]:
print(embedding)

In [ ]:

index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

docsearch = PC(index_name, embeddings)


AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'startswith'

#**Step 09: Similarity Search**

In [ ]:
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)

print(len(texts[21].page_content))
print(len(texts[22].page_content))

results = nlp(" company is going to boom in next year")
print(results)

ModuleNotFoundError: No module named 'llama_index'

In [ ]:
  texts[1100].page_content

'-Paragraph 1\n-Subparagraph (SX 210.4-08(h)(2))\n-URI http://asc.fasb.org/extlink&oid=26873400&loc=d3e23780-122690+ Details\nName:\nus-gaap_IncomeTaxReconciliationStateAndLocalIncomeTaxes\nNamespace Prefix:\nus-gaap_\nData Type:\nxbrli:monetaryItemType\nBalance Type:\ndebit\nPeriod Type:\nduration\nX- Definition\nAmount of the difference between reported income tax expense (benefit) and expected income tax expense (benefit) computed by applying the domestic federal statutory income tax rates to pretax income (loss) from continuing operations attributable to research tax credit.+ References\nReference 1: http://www.xbrl.org/2003/role/presentationRef\n-Publisher FASB\n-Name Accounting Standards Codification\n-Topic 740\n-SubTopic 10\n-Section 50\n-Paragraph 12\n-URI http://asc.fasb.org/extlink&oid=84230637&loc=d3e32687-109319\nReference 2: http://www.xbrl.org/2003/role/presentationRef\n-Publisher FASB\n-Name Accounting Standards Codification\n-Topic 740\n-SubTopic 10\n-Section 50\n-Para

In [ ]:
query = "YOLOv7 outperforms which models"

In [ ]:
# res = index.query(vector = embedding, top_k=5, include_metadata=True, query = query)

docs =  embedding.similarity_search(query, k=4)

In [ ]:
docs

#**Step 10: Creating a Llama2 Model Wrapper**

In [ ]:
from huggingface_hub import notebook_login
import torch

In [ ]:
notebook_login()


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                            #  torch_dtype=torch.float16,
                                            #  use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  model_kwargs={"torch_dtype": torch.float16}
                                             )

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})


In [ ]:
!pip install llama-index-embeddings-langchain

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#**Step 11: Create a Prompt Template**

In [ ]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""


In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [ ]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [ ]:
instruction = """
{context}

Question: {question}
"""

In [ ]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [ ]:
template

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<</SYS>>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
index=VectorStoreIndex.from_documents(texts[1])

NameError: name 'VectorStoreIndex' is not defined

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=embedding.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

NameError: name 'embedding' is not defined

In [ ]:
result = qa_chain("YOLOv7 is used for")

In [ ]:
result['result']

In [ ]:
while True:
    user_input=input(f"prompt:")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa_chain({'query':user_input})
    print(f"Answer:{result['result']}")

prompt:comany name?


NameError: name 'qa_chain' is not defined

In [ ]:
result = qa_chain("company name?")
result['result']
while True:
    user_input=input(f"prompt:")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa_chain({'query':user_input})
    print(f"Answer:{result['result']}")